# Installing and import of libraries

In [ ]:
!pip install icrawler # for web scraping
!pip install pillow # for image manipulation
!pip install opencv-python # for image manipulation
!pip install timm scikit-learn # for the model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import os
from icrawler.builtin import GoogleImageCrawler
from icrawler.builtin import BingImageCrawler
from google.colab import drive
import random
import numpy as np
from PIL import Image, ImageOps, ImageFilter
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score
import timm
from tqdm import tqdm
import itertools
import time
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.metrics import confusion_matrix

Checking CUDA

In [ ]:
import torch

# Check if CUDA is available
print("Is CUDA available?:", torch.cuda.is_available())

# Number of available GPUs
print("Number of GPUs:", torch.cuda.device_count())

# Device name
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("Current device:", torch.cuda.current_device())
else:
    print("No GPU - running on CPU.")




```
# Drive mount
```

Zamontowanie dysku

In [ ]:
drive.mount('/content/drive')


# Data download and processing

Creating folders in Google Drive

In [ ]:
# Base path
base_path = "/content/drive/MyDrive/DL"

# List of folder names
folders = [
    "BT_glioma",
    "BT_meningioma",
    "BT_pituitary",
    "MS",
    "Normal",
]

# Creating folders
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    os.makedirs(folder_path, exist_ok=True)

print("Folders have been created")


In [ ]:
import os
import shutil
from pathlib import Path

# Source and destination paths
src_base = '/content/drive/MyDrive/DL/MRI_scans'
dst_base = '/content/drive/MyDrive/DL/Brain_scans'

# List of folders/categories
categories = ['BT_glioma', 'BT_meningioma', 'BT_pituitary', 'MS', 'Normal']

# Creating destination folder and subfolders
os.makedirs(dst_base, exist_ok=True)

for category in categories:
    src_folder = os.path.join(src_base, category)
    dst_folder = os.path.join(dst_base, category)
    os.makedirs(dst_folder, exist_ok=True)

    # Get list of files (only files, not folders)
    files = [f for f in os.listdir(src_folder) if os.path.isfile(os.path.join(src_folder, f))]

    # Copy up to 300 files
    for file_name in files[:300]:
        src_file = os.path.join(src_folder, file_name)
        dst_file = os.path.join(dst_folder, file_name)
        shutil.copy2(src_file, dst_file)

    print(f'Copied {min(300, len(files))} files from {category}')


Skopiowano 300 plików z BT_glioma
Skopiowano 300 plików z BT_meningioma
Skopiowano 300 plików z BT_pituitary
Skopiowano 300 plików z MS
Skopiowano 300 plików z Normal


For each folder containing images of a specific brain tumor, automatic dataset augmentation is performed.
The goal is to reach a target number of images (e.g., 200) by creating synthetic examples from existing ones.
To achieve this, random (more precisely: pseudorandom) transformations are applied to selected images.

Transformations include:

- Mirror flip – horizontal flip of the image (ImageOps.mirror),

- Rotation – rotation by a random angle chosen from: 90°, 180°, or 270°,

- Cropping with rescaling – removing 10% of the edges from each side and rescaling to the original size,

- Color quantization – reducing the number of colors (randomly between 4–16),

- Gaussian blur – simulating blur with a filter of random radius (1–3 px),

- Gaussian noise – adding random noise to the image (simulating distortions),

- Shear – distorting the image by horizontal shearing,

- Random crop with rescaling – cropping a random region covering 80% of the image and rescaling it back to the original size.

In [ ]:
import os
import random
import numpy as np
import cv2
from PIL import Image, ImageOps, ImageFilter
import uuid

# Path to the folder
folder_path = "/content/drive/MyDrive/DL/Brain_scans/Normal"

# Target number
target_count = 350

# Possible transformations
def apply_random_transformation(img):
    transformation = random.choice([
        'mirror', 'rotate', 'crop', 'quantize',
        'blur', 'noise', 'shear', 'random_crop'
    ])

    if transformation == 'mirror':
        return ImageOps.mirror(img)

    elif transformation == 'rotate':
        angle = random.choice([90, 180, 270])
        return img.rotate(angle)

    elif transformation == 'crop':
        width, height = img.size
        left = width * 0.1
        top = height * 0.1
        right = width * 0.9
        bottom = height * 0.9
        return img.crop((left, top, right, bottom)).resize((width, height))

    elif transformation == 'quantize':
        return img.quantize(colors=random.randint(4, 16)).convert('RGB')

    elif transformation == 'blur':
        return img.filter(ImageFilter.GaussianBlur(radius=random.uniform(1, 3)))

    elif transformation == 'noise':
        img_np = np.array(img)
        noise = np.random.normal(0, 25, img_np.shape).astype(np.uint8)
        noisy = cv2.add(img_np, noise)
        return Image.fromarray(noisy)

    elif transformation == 'shear':
        width, height = img.size
        m = 0.2  # shearing factor
        xshift = abs(m) * width
        new_width = width + int(round(xshift))
        return img.transform((new_width, height), Image.AFFINE,
                             (1, m, -xshift if m > 0 else 0, 0, 1, 0),
                             Image.BICUBIC)

    elif transformation == 'random_crop':
        width, height = img.size
        crop_size = int(min(width, height) * 0.8)
        left = random.randint(0, width - crop_size)
        top = random.randint(0, height - crop_size)
        right = left + crop_size
        bottom = top + crop_size
        return img.crop((left, top, right, bottom)).resize((width, height))

    return img

# Main loop
existing_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))]
image_count = len(existing_files)

# List of original images for augmentation
base_images = [os.path.join(folder_path, f) for f in existing_files]

if not base_images:
    print("No original images available for transformation.")
else:
    while image_count < target_count:
        img_path = random.choice(base_images)
        img = Image.open(img_path).convert('RGB')

        transformed = apply_random_transformation(img)
        new_filename = f"{uuid.uuid4().hex}.jpg"
        transformed.save(os.path.join(folder_path, new_filename))

        image_count += 1

    print(f"Completed – the folder now contains {image_count} images.")


# Model

Helper function to save files in CSV

In [ ]:
import csv
import os

# Function to initialize the CSV file (with header)
def init_csv(output_dir):
    os.makedirs(output_dir, exist_ok=True)
    csv_file = os.path.join(output_dir, "training_metrics.csv")
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                "Epoch", "Epoch Time (s)",
                "Train Accuracy", "Train Loss", "Train F1",
                "Validation Accuracy", "Validation Loss", "Validation F1"
            ])
    return csv_file  # <- return the path to the file

# Function to log metrics after each epoch
def log_metrics(output_dir, epoch, epoch_time,
                train_acc, train_loss, train_f1,
                val_acc, val_loss, val_f1):

    csv_file = os.path.join(output_dir, "training_metrics.csv")

    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([
            epoch, f"{epoch_time:.2f}",
            f"{train_acc:.4f}", f"{train_loss:.4f}", f"{train_f1:.4f}",
            f"{val_acc:.4f}", f"{val_loss:.4f}", f"{val_f1:.4f}"
        ])


Helper function to plot charts

In [ ]:
def save_epoch_plots(epoch, train_acc, val_acc, train_loss, val_loss, model, val_loader, class_names, device, output_dir):
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)

    epochs_range = range(1, len(history['train_acc']) + 1
)

    # Wykres Accuracy
    plt.figure()
    plt.plot(epochs_range, history['train_acc'], label='Train Accuracy')
    plt.plot(epochs_range, history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy per Epoch')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f"accuracy_epoch_{epoch+1}.png"))
    plt.close()

    # Wykres Loss
    plt.figure()
    plt.plot(epochs_range, history['train_loss'], label='Train Loss')
    plt.plot(epochs_range, history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss per Epoch')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f"loss_epoch_{epoch+1}.png"))
    plt.close()

    # Macierz pomyłek
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (Epoch {epoch+1})')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"confusion_matrix_epoch_{epoch+1}.png"))
    plt.close()


Training and validation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# ======================== DATA PATH
data_path = "/content/drive/MyDrive/DL/Brain_scans/"

# ======================== TRANSFORMATIONS
"""
Selecting the computation device (GPU if available, otherwise CPU),
setting the path to the folder with MRI data, and preparing image transformations.
Images are resized to 224x224 pixels, converted to tensors, and normalized to the range [-1, 1].
"""

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


# ======================== LOADING DATA
"""
Loading the entire image dataset from subfolders, where each folder represents one class.
Images are automatically labeled based on folder names and transformed according to the defined transformations.
Extracting class names and their count for further processing/modeling.
"""

full_dataset = ImageFolder(root=data_path, transform=transform)
class_names = full_dataset.classes
num_classes = len(class_names)


# ======================== GRID SEARCH PARAMS
"""
Defining lists of potential hyperparameter values: learning rate, dropout rate, and batch size.
Using itertools to create the full grid of parameter combinations (grid search),
which will allow finding the optimal set for best model performance.
"""
learning_rates = [0.001, 0.0001]
drop_rates = [0.3, 0.5]
batch_sizes = [16, 32]

# ======================== All hyperparameter combinations
combinations = list(itertools.product(learning_rates, drop_rates, batch_sizes))


"""
Main loop performing hyperparameter grid search.
For each combination, the dataset is randomly split into train/val/test (70/15/15),
new DataLoaders are created with the current batch size, and a new ConvNeXt-Atto model
is initialized with the corresponding dropout rate.
This allows evaluating how different settings affect model performance.
"""


for lr, drop_rate, batch_size in combinations:
    print(f"\n========================> Training: lr={lr}, drop_rate={drop_rate}, batch_size={batch_size}")

    # ======================== DATA SPLIT EACH TIME (for randomness)
    """
    Randomly splitting the dataset into training, validation, and test sets according to the predefined sizes.
    Creating data loaders with the specified batch size (16) and parallel loading (2 workers).
    Training data is shuffled each epoch, while validation and test data maintain a fixed order.
    """

    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # ======================== MODEL
    """
    Creating a ConvNeXt-Atto model from the timm library.
    Model initialized with weights pretrained on ImageNet (transfer learning),
    adjusted to the number of classes in the task.
    Dropout and DropPath are set dynamically to test their effect during grid search.
    Model is moved to the available device (GPU or CPU).
    """

    model = timm.create_model(
        'convnext_atto',
        pretrained=True,
        num_classes=num_classes,
        drop_rate=drop_rate,
        drop_path_rate=0.2
    )
    model.to(device)

    # ======================== FREEZING FEATURE EXTRACTOR
    """
    Freezing all layers of the ConvNeXt model except the classification layer.
    This allows only the last layer's weights (head) to be updated during training,
    adapting the model to the specific task (e.g., tumor classification)
    while keeping the features pretrained on ImageNet.
    """

    for param in model.parameters():
        param.requires_grad = False
    for param in model.get_classifier().parameters():
        param.requires_grad = True


    # Only part of the earlier layers:

    # for name, param in model.named_parameters():
    # if "blocks.29" in name or "head" in name:  # e.g., last block + head
    #     param.requires_grad = True


    # To train the entire model:

    # for param in model.parameters():
    #     param.requires_grad = True


    # ======================== LOSS, OPTIMIZER
    """
    Initializing Adam optimizer for the model's classification layer.
    You can also use optim.SGD(..., momentum=0.9) or optim.AdamW(...)  # Adam with better regularization
    Then, defining an `evaluate` function that performs model evaluation
    on a given dataset (validation or test).
    Returns average loss, accuracy, and weighted F1-score.
    Turning off gradient tracking (`torch.no_grad()`) speeds up computation and reduces memory usage.
    """

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.get_classifier().parameters(), lr=lr)

    def evaluate(loader):
      model.eval()
      total_loss = 0
      all_preds, all_labels = [], []

      with torch.no_grad():
          for images, labels in loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              loss = criterion(outputs, labels)
              total_loss += loss.item()

              _, preds = torch.max(outputs, 1)
              all_preds.extend(preds.cpu().numpy())
              all_labels.extend(labels.cpu().numpy())

      acc = accuracy_score(all_labels, all_preds)
      f1 = f1_score(all_labels, all_preds, average='weighted')
      avg_loss = total_loss / len(loader)

      return avg_loss, acc, f1



    # ======================== TRAINING ========================
    """
    Main training loop for the model over a specified number of epochs.

    - Sets the model to training mode.
    - Iterates over batches, performing forward and backward passes and updating weights.
    - Collects and calculates metrics (loss, accuracy, F1) for training.
    - After each epoch, calls the evaluation function on the validation set.
    - Prints training and validation results.
    - Creates a directory to save results (models, plots) with a name based on hyperparameters.
    """

    history = {
    'train_acc': [],
    'val_acc': [],
    'train_loss': [],
    'val_loss': []
                      }

    epochs = 10
    for epoch in range(epochs):
        start_time = time.time()

        model.train()
        total_loss = 0
        all_preds, all_labels = [], []

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        epoch_time = time.time() - start_time

        train_acc = accuracy_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds, average='weighted')
        val_loss, val_acc, val_f1 = evaluate(val_loader)

        print(f"\nEpoch {epoch+1}")
        print(f"Train loss: {total_loss / len(train_loader):.4f}, acc: {train_acc:.4f}, f1: {train_f1:.4f}")


Używane urządzenie: cuda

========================> Trenowanie: lr=0.001, drop_rate=0.3, batch_size=16


Epoch 1/10: 100%|██████████| 77/77 [00:07<00:00, 10.61it/s]



Epoch 1
Train loss: 1.1271, acc: 0.5527, f1: 0.5484
Val   loss: 0.7290, acc: 0.8015, f1: 0.7994


Epoch 2/10: 100%|██████████| 77/77 [00:07<00:00, 10.66it/s]



Epoch 2
Train loss: 0.7853, acc: 0.7061, f1: 0.7027
Val   loss: 0.5957, acc: 0.8206, f1: 0.8258


...


Epoch 8
Train loss: 1.1375, acc: 0.5600, f1: 0.5552
Val   loss: 0.9705, acc: 0.6985, f1: 0.6856


Epoch 9/10: 100%|██████████| 39/39 [00:06<00:00,  6.08it/s]



Epoch 9
Train loss: 1.1220, acc: 0.5682, f1: 0.5649
Val   loss: 0.9344, acc: 0.7214, f1: 0.7124


Epoch 10/10: 100%|██████████| 39/39 [00:06<00:00,  5.66it/s]



Epoch 10
Train loss: 1.0730, acc: 0.5902, f1: 0.5882
Val   loss: 0.9026, acc: 0.7252, f1: 0.7179

== TEST SET dla lr=0.0001, drop_rate=0.5, batch_size=32 ==
Test loss: 0.9734, acc: 0.7186, f1: 0.7159
